In [1]:
import requests
from bs4 import BeautifulSoup as soup
from multiprocessing import Pool

import MyFunctionsList as mfs
import os
import time
import datetime
import pandas as pd

from DoNotPublish import password
from sqlalchemy import create_engine
from bs4 import BeautifulSoup as soup

# Debugging
from IPython.display import display, clear_output

In [2]:
protocol = 'postgresql'
username = 'postgres'
password = password
host = 'localhost'
port = 5432
database_name = 'pro_clubs_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [79]:
# existing_sql_data = pd.read_sql_query('select * from clubs', con=engine)
# existing_sql_data.active = 'No'
# existing_sql_data.head()

existing_sql_data = pd.read_csv('raw_scraped_data/club_list_20592.csv')
existing_sql_data.active = 'No'
existing_sql_data.head()

,pull_time,club_name,active,console,club_id,division,ten_match_record,total_games,wins,losses,...,best_season_div,best_season_points,region,stadium,total_rank_points,league_points,cup_points,div_1_titles,other_div_titles,total_trophies
0,2023-03-21 00:18:41,Bang Average FC,No,xboxone,1940614,1,WWWWWWLLDD,510,393,62,...,1,25,British Isles,San Siro,14862,14862,0,32,8,40
1,2023-03-21 00:18:41,FootieXI,No,xboxone,4339359,1,WWWWWWWWWL,507,407,51,...,1,25,British Isles,Old Trafford,16257,16257,0,39,9,48
2,2023-03-21 00:18:41,NO TRUENES,No,xboxone,3705104,1,WWWWWWWWLD,503,419,51,...,1,25,Central America,ESTADIO DE TRONACION,16668,16668,0,42,9,51
3,2023-03-21 00:18:41,rock club,No,xboxone,542709,1,WWWWWWLLDD,1260,864,183,...,1,25,British Isles,Parc des Princes,32806,32806,0,70,5,75
4,2023-03-21 00:18:41,XVC AC MILAN,No,xboxone,10205899,1,WWWWWWWWLD,328,270,21,...,1,25,British Isles,Old Trafford,9773,9773,0,21,9,30


In [4]:
# Debug
start_time = time.time()
counter = 0
length = len(existing_sql_data)

matches = []
player_performances = []
opponents_list = []

# Extract the make data, player performance data, and opponent data for all clubs in DB. 
for index, row in existing_sql_data.iterrows():
    
    console = row['console']
    club_id = row['club_id']
    club_name = row['club_name']
    
    match_data, individual_player_perform_data, opponents = mfs.last_five_match_scrape(console, club_id, club_name)
    matches.extend(match_data)
    player_performances.extend(individual_player_perform_data)
    opponents_list.extend(opponents)
    
    clear_output(wait=True)
    counter += 1
    print(f"{counter}/{length}")
    
# Debug
end_time = time.time()
print("Time taken:", end_time - start_time, "seconds")

7907/7907
Time taken: 11800.111170768738 seconds


In [5]:
#Generates and stores current time for later use. 
current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [6]:
# Turn obtained player performance data in a df, add a time column indicting when it was pulled. 
player_performances_df = pd.DataFrame(player_performances)
player_performances_df.insert(0, 'pull_time', current_time)

# Pull all current player performance data. 
# existing_player_performance = pd.read_sql_query('select * from single_match_player_performance', con=engine)
existing_player_performance = pd.read_csv('final_player_perform_74078.csv')

# Merge this with the  recently obtained performance data, then drop any duplicate rows. 
player_performances_df = player_performances_df.append(existing_player_performance)
player_performances_df.drop_duplicates(inplace=True)

# Stores the Data
player_performances_df.to_sql('single_match_player_performance', engine, if_exists='replace', index=False)

# (Optional) Output CSV
# player_performances_df.to_csv('player_performances_df.csv', index=False)

In [7]:
# Turns obtained opponent list into a df. 
opponents_list_df = pd.DataFrame(opponents_list)

# Merges with existing clubs_df, drops duplicate clubs. 
clubs_to_scrape_df = opponents_list_df.append(existing_sql_data[['console', 'club_name', 'club_id']])
clubs_to_scrape_df.drop_duplicates(inplace=True)

clubs_to_scrape_df.head()

,console,club_name,club_id
0,xboxone,Parkgate United,15695788
1,xboxone,All The Niguez,18834
2,xboxone,punjabi panther,7612049
3,xboxone,ieuan and shane,176694
4,xboxone,fc CoolTeam,8369654


In [8]:
# Debug
start_time = time.time()
length  = len(clubs_to_scrape_df)
counter = 0
deleted_counter = 0

scaped_club_list = []

for index, row in clubs_to_scrape_df.iterrows():
    console = row['console']
    club_id = row['club_id']
    club_data = mfs.club_scrape(console, club_id)
    if club_data['division'] > 0:
        scaped_club_list.append(club_data)
    if club_data['division'] == 0:
        deleted_counter += 1

    #debug
    clear_output(wait=True)
    counter += 1
    print(f"{counter}/{length}.  {deleted_counter} clubs have been deleted")
        
# Debug
end_time = time.time()
print("Time taken:", end_time - start_time, "seconds")

20785/20785.  192 clubs have been deleted
Time taken: 12977.680989742279 seconds


In [80]:
# Converts scraped club list to DF. 
scraped_club_df = pd.DataFrame(scaped_club_list)
scraped_club_df
# # Adds time column. 
scraped_club_df.insert(0, 'pull_time', current_time)

# Updates existing rows in the club_df. 
scraped_club_df.set_index(['club_id'], inplace = True)
existing_sql_data.set_index(['club_id'], inplace = True)
existing_sql_data.update(scaped_club_df)
final_club_df = pd.concat([existing_sql_data, scraped_club_df[~scraped_club_df.index.isin(existing_sql_data.index)]])
final_club_df.reset_index(inplace = True)
final_club_df = final_club_df[['pull_time', 'club_name', 'active', 'console', 'club_id', 'division',
       'ten_match_record', 'total_games', 'wins', 'losses', 'draws',
       'goals_scored', 'goals_per_match', 'goals_conceded',
       'goals_conceded_per_match', 'goal_difference', 'promotions', 'holds',
       'relegations', 'best_season_div', 'best_season_points', 'region',
       'stadium', 'total_rank_points', 'league_points', 'cup_points',
       'div_1_titles', 'other_div_titles', 'total_trophies']]

scraped_club_df = pd.DataFrame(scaped_club_list)
final_club_df.update(scaped_club_df)

In [83]:
print(len(final_club_df))
print(final_club_df.club_id.nunique())

20598
20592


In [16]:
# existing_matches = pd.read_sql_query('select * from games', con=engine)
existing_matches = pd.read_csv('match_list_10065.csv')

matches_df = pd.DataFrame(matches)
matches_df.insert(0, 'pull_time', current_time)

matches_df = matches_df.append(existing_matches)
final_matches_df = matches_df.drop_duplicates(subset=['game_id'])

In [17]:
# Pull all current player performance data. 
existing_player_performance = pd.read_sql_query('select * from single_match_player_performance', con=engine)
# existing_player_performance = pd.read_csv('final_player_perform_30387.csv')

player_performances_df = pd.DataFrame(player_performances)
player_performances_df.insert(0, 'pull_time', current_time)


# Merge this with the  recently obtained performance data, then drop any duplicate rows. 
player_performances_df = player_performances_df.append(existing_player_performance)
final_player_perform_df = player_performances_df.drop_duplicates()

In [84]:
final_club_df.drop(columns=['pull_time'], inplace = True)
final_club_df.drop_duplicates(inplace=True)
final_club_df.insert(0, 'pull_time', current_time)
final_club_df.to_csv(f'raw_scraped_csv/club_list_{len(final_club_df)}.csv', index = False)


final_matches_df.drop(columns=['pull_time'], inplace = True)
final_matches_df.drop_duplicates(inplace=True)
final_matches_df.insert(0, 'pull_time', current_time)
final_matches_df.to_csv(f'raw_scraped_csv/match_list_{len(final_matches_df)}.csv', index = False)

final_player_perform_df.drop(columns=['pull_time'], inplace = True)
final_player_perform_df.drop_duplicates(inplace=True)
final_player_perform_df.insert(0, 'pull_time', current_time)
final_player_perform_df.to_csv(f'raw_scraped_csv/final_player_perform_{len(final_player_perform_df)}.csv', index=False)

C:\Users\cjrad\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\cjrad\Anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [85]:
len(final_player_perform_df[final_player_perform_df.position == 'goalkeeper'])

8979